# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 29 2022 10:41AM,254001,21,637105,"NBTY Global, Inc.",Released
1,Dec 29 2022 10:41AM,253997,10,8959680,Eywa Pharma Inc.,Released
2,Dec 29 2022 10:41AM,253997,10,8959681,Eywa Pharma Inc.,Released
3,Dec 29 2022 10:41AM,253997,10,8959682,Eywa Pharma Inc.,Released
4,Dec 29 2022 10:41AM,253997,10,8959684,Eywa Pharma Inc.,Released
5,Dec 29 2022 10:41AM,253997,10,8959687,Eywa Pharma Inc.,Released
6,Dec 29 2022 10:41AM,253997,10,8959688,Eywa Pharma Inc.,Released
7,Dec 29 2022 10:41AM,253997,10,8959696,Eywa Pharma Inc.,Released
8,Dec 29 2022 10:41AM,253997,10,8959697,Eywa Pharma Inc.,Released
9,Dec 29 2022 10:41AM,253997,10,8961436,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,253997,Released,12
39,253998,Released,1
40,253999,Released,1
41,254000,Released,1
42,254001,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253997,NaN,12.0
253998,NaN,1.0
253999,NaN,1.0
254000,NaN,1.0
254001,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253905,0.0,11.0
253908,0.0,1.0
253909,0.0,3.0
253913,0.0,3.0
253916,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253905,0,11
253908,0,1
253909,0,3
253913,0,3
253916,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253905,0,11
1,253908,0,1
2,253909,0,3
3,253913,0,3
4,253916,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253905,,11
1,253908,,1
2,253909,,3
3,253913,,3
4,253916,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc."
1,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.
13,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc."
14,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.
15,Dec 29 2022 10:38AM,253998,10,"Carwin Pharmaceutical Associates, LLC"
16,Dec 29 2022 10:34AM,253995,10,"Methapharm, Inc."
36,Dec 29 2022 10:31AM,253996,19,"NAPP Technologies, LLC"
37,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc."
43,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc."
44,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",,1
1,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,,12
2,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",,1
3,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.,,1
4,Dec 29 2022 10:38AM,253998,10,"Carwin Pharmaceutical Associates, LLC",,1
5,Dec 29 2022 10:34AM,253995,10,"Methapharm, Inc.",,20
6,Dec 29 2022 10:31AM,253996,19,"NAPP Technologies, LLC",,1
7,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",,6
8,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc.",,1
9,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",1,
1,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,12,
2,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",1,
3,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.,1,
4,Dec 29 2022 10:38AM,253998,10,"Carwin Pharmaceutical Associates, LLC",1,
5,Dec 29 2022 10:34AM,253995,10,"Methapharm, Inc.",20,
6,Dec 29 2022 10:31AM,253996,19,"NAPP Technologies, LLC",1,
7,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",6,
8,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc.",1,
9,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",1,
1,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,12,
2,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",1,
3,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.,1,
4,Dec 29 2022 10:38AM,253998,10,"Carwin Pharmaceutical Associates, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",1.0,NaN
1,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,12.0,NaN
2,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",1.0,NaN
3,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.,1.0,NaN
4,Dec 29 2022 10:38AM,253998,10,"Carwin Pharmaceutical Associates, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",1.0,0.0
1,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,12.0,0.0
2,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",1.0,0.0
3,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.,1.0,0.0
4,Dec 29 2022 10:38AM,253998,10,"Carwin Pharmaceutical Associates, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,7872499,187.0,0.0
16,253985,0.0,1.0
17,1015904,5.0,0.0
19,761961,3.0,0.0
20,507963,4.0,0.0
21,761917,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,7872499,187.0,0.0
1,16,253985,0.0,1.0
2,17,1015904,5.0,0.0
3,19,761961,3.0,0.0
4,20,507963,4.0,0.0
5,21,761917,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,187.0,0.0
1,16,0.0,1.0
2,17,5.0,0.0
3,19,3.0,0.0
4,20,4.0,0.0
5,21,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,187.0
1,16,Released,0.0
2,17,Released,5.0
3,19,Released,3.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,19,20,21
Status,,,,,,
Executing,0.0,1.0,0.0,0.0,0.0,1.0
Released,187.0,0.0,5.0,3.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,19,20,21
0,Executing,0.0,1.0,0.0,0.0,0.0,1.0
1,Released,187.0,0.0,5.0,3.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,19,20,21
0,Executing,0.0,1.0,0.0,0.0,0.0,1.0
1,Released,187.0,0.0,5.0,3.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()